# BasicAssessMetrics

In [57]:
%%bash
curr_path = `pwd`
cd opt
git clone https://github.com/RNA-Puzzles/BasicAssessMetrics.git

Cloning into 'BasicAssessMetrics'...


u'/Users/magnus/work-src/rna-pdb-tools/opt'

In [61]:
import sys
import os

import pdb_utils
import utils
import extract

from operator import attrgetter

RESIDUES_LIST = "data/residues.list"
ATOMS_LIST = "data/atoms.list"

def CleanFormat(f):
    """
    CleanFormat is a function used to format different platform formats to unix. Users need to install dos2unix
    """
    os.system( "mac2unix -q %s" %f )
    os.system( "dos2unix -q %s" %f )

In [62]:
def normalize_structure(struct, out_file = None, index_file=None, extract_file = None):
    pdb_normalizer = pdb_utils.PDBNormalizer( RESIDUES_LIST, ATOMS_LIST )
    ok = pdb_normalizer.parse( struct, out_file )
    if not ok:
        sys.stderr.write("ERROR: structure not normalized!\n")
    else:
        sys.stderr.write("INFO: Normalization succeded!\n")
    if not extract_file is None:
        coords=open(index_file).read()
        extract.extract_PDB(SOLUTION_NORMAL,coords, extract_file)
        sys.stderr.write("INFO:	structure extracted\n")

In [63]:
# PVALUE set according to Hajdin et al., RNA (7) 16, 2010, either "+" or "-"
def calc_RMSD(native_file, native_index, prediction_file, prediction_index, PVALUE = "-"):
    res_struct = pdb_utils.PDBStruct()
    res_struct.load( native_file, native_index )
    res_raw_seq = res_struct.raw_sequence()

    sol_struct = pdb_utils.PDBStruct()
    sol_struct.load( prediction_file, prediction_index )
    sol_raw_seq = sol_struct.raw_sequence()

    if( sol_raw_seq != res_raw_seq ):
        sys.stderr.write("ERROR Result sequence != Solution sequence!\n")
        sys.stderr.write("DATA Solution sequence --> '%s'\n" %sol_raw_seq )
        sys.stderr.write("DATA Result sequence   --> '%s'\n" %res_raw_seq )
        return(-1)
    # computes the RMSD
    comparer = pdb_utils.PDBComparer()
    rmsd = comparer.rmsd( sol_struct, res_struct )
    sys.stderr.write("INFO Partial RMSD --> %f\n" %rmsd )
    pvalue = comparer.pvalue( rmsd, len(sol_raw_seq), PVALUE )
    sys.stderr.write("INFO Partial P-Value --> %e\n" %pvalue )
    return(rmsd, pvalue)

In [10]:
def InteractionNetworkFidelity(native_file, native_index, prediction_file, prediction_index):
    res_struct = pdb_utils.PDBStruct()
    res_struct.load( native_file, native_index )
    res_raw_seq = res_struct.raw_sequence()

    sol_struct = pdb_utils.PDBStruct()
    sol_struct.load( prediction_file, prediction_index )
    sol_raw_seq = sol_struct.raw_sequence()

    if( sol_raw_seq != res_raw_seq ):
        sys.stderr.write("ERROR Result sequence != Solution sequence!\n")
        sys.stderr.write("DATA Solution sequence --> '%s'\n" %sol_raw_seq )
        sys.stderr.write("DATA Result sequence	 --> '%s'\n" %res_raw_seq )
        return(-1)
    # computes the RMSD
    comparer = pdb_utils.PDBComparer()
    rmsd = comparer.rmsd( sol_struct, res_struct )
    INF_ALL = comparer.INF( sol_struct, res_struct, type="ALL" )
    DI_ALL = rmsd / INF_ALL
    INF_WC = comparer.INF( sol_struct, res_struct, type="PAIR_2D" )
    INF_NWC = comparer.INF( sol_struct, res_struct, type="PAIR_3D" )
    INF_STACK = comparer.INF( sol_struct, res_struct, type="STACK" )
    return (rmsd,DI_ALL, INF_ALL, INF_WC, INF_NWC,INF_STACK)

In [41]:
# Normalize PDB format, correct residue names and atom names. 
normalize_structure('example/14_solution_0.pdb','example/14_solution_normalized.pdb')

# calculate RMSD for RNA structures
# require biopython
rmsd, pvalue = calc_RMSD("example/14_solution_0.pdb",
        "example/14_solution_0.index",
        "example/14_ChenPostExp_2.pdb",
        "example/14_ChenPostExp_2.index")

print '14_ChenPostExp_2'
print '  RMSD:', rmsd
print '  pvalue:', pvalue

# calculate InteractionNetworkFidelity and Deformation Index for RNA structures
# need to have MA-annotate in the directory or set in mcannotate.py
rmsd, DI_ALL, INF_ALL, INF_WC, INF_NWC,INF_STACK = InteractionNetworkFidelity("example/14_solution_0.pdb",
          "example/14_solution_0.index",
          "example/14_ChenPostExp_2.pdb",
          "example/14_ChenPostExp_2.index")

print '14_ChenPostExp_2, rmsd', rmsd
print "  DI_ALL:", DI_ALL
print "  INF_ALL:", INF_ALL

INFO: Normalization succeded!
WARNING    >> Atom P from residue (' ', 1, ' ') not found in target atom list
WARNING    >> Atom OP1 from residue (' ', 1, ' ') not found in target atom list
WARNING    >> Atom OP2 from residue (' ', 1, ' ') not found in target atom list
INFO Partial RMSD --> 7.751173
INFO Partial P-Value --> 7.327472e-15
WARNING    >> Atom P from residue (' ', 1, ' ') not found in target atom list
WARNING    >> Atom OP1 from residue (' ', 1, ' ') not found in target atom list
WARNING    >> Atom OP2 from residue (' ', 1, ' ') not found in target atom list


14_ChenPostExp_2
  RMSD: 7.751173243045827
  pvalue: 7.327471962526033e-15
14_ChenPostExp_2, rmsd 7.751173243045827
  DI_ALL: 10.643784178530254
  INF_ALL: 0.72823472489


# Deformation Profile

Read [DeformationProfile Manual.pdf](https://github.com/RNA-Puzzles/DeformationProfile/blob/master/manual_1_0_1.pdf) for more details. 

In [91]:
%%bash
git clone https://github.com/RNA-Puzzles/DeformationProfile.git

Cloning into 'DeformationProfile'...


## Comparing two similar models

In [92]:
cd DeformationProfile/examples/ex1
cd /Users/magnus/work-src/rna-pdb-tools/opt/DeformationProfile

SyntaxError: invalid syntax (<ipython-input-92-aaa236dd1b63>, line 1)

In [93]:
%%bash
python dp.py examples/ex1/a.pdb examples/ex1/b.pdb

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
reference:    a.pdb
comparing:    b.pdb

ref id:       201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 
cmp id:       201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 

ref chain:      B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B 
cmp chain:      B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B 
ref residues:   G   G   A   C   U   A   G   C   G   G   A   G   G   C   U   A   G   U   C   C 
cmp residues:   G   G   A   C   U   A   G   C   G   G   A   G   G   C   U   A   G   U   C   C 
align. index:   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19 
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 


opening reference file: 'examples/ex1/a.pdb'
opening comparing file: 'examples/ex1/b.pdb'
comparing models...
step: 0 of 19step: 1 of 19step: 2 of 19step: 3 of 19step: 4 of 19step: 5 of 19step: 6 of 19step: 7 of 19step: 8 of 19step: 9 of 19step: 10 of 19step: 11 of 19step: 12 of 19step: 13 of 19step: 14 of 19step: 15 of 19

![x](opt/DeformationProfile/examples/ex1/b.svg)

In [94]:
%%bash
head -n 15 examples/ex1/b.dat

#DP 1.0
REF_PDB	examples/ex1/a.pdb
REF_MODEL	0
REF_MODEL_SEQUENCE	GGACUAGCGGAGGCUAGUCC
REF_MODEL_RESIDUES	(B:201:'G  ')(B:202:'G  ')(B:203:'A  ')(B:204:'C  ')(B:205:'U  ')(B:206:'A  ')(B:207:'G  ')(B:208:'C  ')(B:209:'G  ')(B:210:'G  ')(B:211:'A  ')(B:212:'G  ')(B:213:'G  ')(B:214:'C  ')(B:215:'U  ')(B:216:'A  ')(B:217:'G  ')(B:218:'U  ')(B:219:'C  ')(B:220:'C  ')
CMP_PDB	examples/ex1/b.pdb
CMP_MODEL	0
CMP_MODEL_SEQUENCE	GGACUAGCGGAGGCUAGUCC
CMP_MODEL_RESIDUES	(B:201:'G  ')(B:202:'G  ')(B:203:'A  ')(B:204:'C  ')(B:205:'U  ')(B:206:'A  ')(B:207:'G  ')(B:208:'C  ')(B:209:'G  ')(B:210:'G  ')(B:211:'A  ')(B:212:'G  ')(B:213:'G  ')(B:214:'C  ')(B:215:'U  ')(B:216:'A  ')(B:217:'G  ')(B:218:'U  ')(B:219:'C  ')(B:220:'C  ')
LOCAL_RMSD	0.455	0.170	0.093	0.386	0.876	0.140	0.162	0.361	0.782	1.005	0.132	0.488	0.977	0.388	0.338	0.484	0.115	0.164	0.106	0.072
ROW_MEANS	3.090	2.476	2.573	2.873	2.802	1.524	1.490	1.833	4.401	3.858	2.353	2.725	6.356	2.263	1.805	2.301	2.531	2.065	2.437	3.917
COL_MEANS	3.5

## Comparing many similar models in single run

In [103]:
%%bash -s "/Users/magnus/work-src/rna-pdb-tools/opt/DeformationProfile/examples/ex2"
cd $1 
mkdir out
python ../../dp.py -c ex2a.cfg

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
reference:    a.pdb
comparing:    b1.pdb

ref id:       201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 
cmp id:       201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 

ref chain:      B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B 
cmp chain:      B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B 
ref residues:   G   G   A   C   U   A   G   C   G   G   A   G   G   C   U   A   G   U   C   C 
cmp residues:   G   G   A   C   U   A   G   C   G   G   A   G   G   C   U   A   G   U   C   C 
align. index:   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19 
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
reference:    a.pdb
comparing:    b2.pdb

ref id:       201 202 203 204 205 206 207 208 209 210 211 212 213 2

mkdir: out: File exists
opening reference file: './ref/a.pdb'
opening comparing file: './cmp/b1.pdb'
comparing models...
step: 0 of 19step: 1 of 19step: 2 of 19step: 3 of 19step: 4 of 19step: 5 of 19step: 6 of 19step: 7 of 19step: 8 of 19step: 9 of 19step: 10 of 19step: 11 of 19step: 12 of 19step: 13 of 19step: 14 of 19step: 15 of 19

In [105]:
%%bash -s "/Users/magnus/work-src/rna-pdb-tools/opt/DeformationProfile/examples/ex3"
cd $1 
mkdir out
python ../../dp.py -c ex3.cfg

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
reference:    a.pdb
comparing:    b.pdb

ref id:       201 202 203 204 205 206 207 208 209   0   1   2   3   4   5   6   7   8   9  10 
cmp id:         1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20 

ref chain:      B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B   B 
cmp chain:      A   A   A   A   A   A   A   A   A   A   A   A   A   A   A   A   A   A   A   A 
ref residues:   G   G   A   C   U   A   G   C   G   G   A   G   G   C   U   A   G   U   C   C 
cmp residues:   G   G   A   C   U   A   G   C   G   G   A   G   G   C   U   A   G   U   C   C 
align. index:   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19 
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 


mkdir: out: File exists
opening reference file: 'a.pdb'
opening comparing file: 'b.pdb'
comparing models...
step: 0 of 19step: 1 of 19step: 2 of 19step: 3 of 19step: 4 of 19step: 5 of 19step: 6 of 19step: 7 of 19step: 8 of 19step: 9 of 19step: 10 of 19step: 11 of 19step: 12 of 19step: 13 of 19step: 14 of 19step: 15 of 19

# mcq-cli

[MCQ (Mean of Circular Quantities)](https://github.com/tzok/mcq4structures) has been first presented in the paper “MCQ4Structures to compute similarity of molecule structures” by Zok et al. (Central European Journal of Operations Research, 2014;22(3):457-474. doi: 10.1007/s10100-013-0296-5).

[LCS-TA (Longest Continuous Segments in Torsion Angle space)](https://github.com/tzok/mcq4structures) applies a measure first described in the paper “LCS-TA to identify similar fragments in RNA 3D structures” by Wiedemann et al. (BMC Bioinformatics, 2017;18(1):456. doi: 10.1186/s12859-017-1867-6). A full implementation of both methods can be found on github, maintained by Zok and Wiedemann.

Most important features of both methods / measures:

- MCQ
    - The method applies to a pair of 3D structures and is size independent.
    - It can be used for a set of structures in all-against-all or all-against-target mode.
    - It translates typical algebraic representation of a 3D structure into the trigonometric one (a set of torsion angles).
    - It computes the distance between structures in torsion angle space.
    - The distance is measured as mean of local distances between the corresponding angles, and provided in degrees.
    - The measure is sequence independent.

- LCS-TA
    - The method applies to a pair of 3D structures and is size independent.
    - It uses MCQ-based measure for structure comparison.
    - Within the compared structures, it finds the longest continuous segments which display similarity in torsion angle space.
    - Two segments are considered similar if their MCQ is below predefined threshold.
    - The method provides segment length and its position in the structure.
    - The length of the longest continuous segment is a measure of similarity of two structures.
    - The method can be run in sequence dependent or sequence independent mode.

MCQ is maintained by Zok, while LCS-TA is maintained by Wiedemann

In [ ]:
@inprogress

# rnaqua: A tool supporting quality assessment of RNA 3D structures

[RNAQUA (RNA QUality Assessment)](https://github.com/mantczak/rnaqua) is a web service based wrapper of basic RNA comparison metrics. It is a RESTful web service client developed in Java. The tool provides a set of web services initially designed for RNAssess (Lukasiak et al., Nucleic Acids Research, 2015;43(W1):W502-W506. doi:10.1093/nar/gkv557) to support the quality assessment of RNA 3D structures. RNAQUA requires a stable release of JRE 7 (or later) installed on a user workstation with an internet access. Full implementation of the tool is stored on github and maintained by Antczak. The binaries and use cases can be downloaded from here. Most important features of RNAQUA:

- At the input, a user provides RNA 3D structure(s) in PDB format.
- Output data are returned in XML format.
- RNAQUA allows to compute the following measures:
    - ClashScore,
    - Root-mean-square deviation (RMSD),
    - Interaction network fidelity (INF),
    - Deformation index (DI),
    - P-value (either for entire RNA 3D structure(s) or for a set of discontinuous 3D substructures).
- Two processing modes are available:
    - An analysis of single RNA 3D structure (PDB validation, ClashScore, sequence-based analysis, structure unification),
    - An analysis of RNA 3D model(s) with respect to the reference structure (RMSD, INF, DI, P-value, Deformation profile, sequence-based differences between structures, multiple models over the reference structure superposition).
- Optionally, the user can define alignment between the reference structure and all corresponding RNA 3D model(s) which is helpful if there are differences in sequence, distribution of chains or residue numbering.

This metric tool is maintained by Antczak.

**DEMO**

Run rnaqua using Docker.

In [6]:
%%bash 
docker exec -i rnapuzzles_toolkit_1 /home/demo/rna-puzzles-toolkit/rnaqua-binary/bin/rnaqua.sh -h

usage: rnaqua
 -c,--command <arg>   supported commands: PDB-VALIDATION, DEFORMATION-PROFILE, CLASH-SCORE, ROOT-MEAN-SQUARE-DEVIATION, ALL-INTERACTION-NETWORK-FIDELITY-SCORES-AT-ONCE,
                      INTERACTION-NETWORK-FIDELITY-WATSON-CRICK, INTERACTION-NETWORK-FIDELITY-NON-WATSON-CRICK, INTERACTION-NETWORK-FIDELITY-STACKING, INTERACTION-NETWORK-FIDELITY-ALL, P-VALUE,
                      DEFORMATION-INDEX, ALL-SCORES-AT-ONCE, SEQUENCE, FRAGMENT, ORIGINAL-3D, RENUMERATED-3D


## PDB-VALIDATION


In [20]:
%%bash 
docker exec -i rnapuzzles_toolkit_1 /home/demo/rna-puzzles-toolkit/rnaqua-binary/bin/rnaqua.sh \
--command PDB-VALIDATION \
--multiple-models-directory-path rna-puzzles-toolkit/rnaqua-binary/use-cases/SINGLE-STRUCTURE-ANALYSIS/PDB-VALIDATION/models/ \
--output-file-path /tmp/validation.xml
docker exec -i rnapuzzles_toolkit_1 cat /tmp/validation.xml

13 Oct 2018 17:54:20 : Command: PDB-VALIDATION
Base pairs identification tool: MC-ANNOTATE
Consider atoms supported by RNA-Puzzles only: N
Output file path: /tmp/validation.xml
Multiple PDB models directory path: rna-puzzles-toolkit/rnaqua-binary/use-cases/SINGLE-STRUCTURE-ANALYSIS/PDB-VALIDATION/models/
13 Oct 2018 17:54:21 : Command processed properly


In [21]:
%%bash
docker exec -i rnapuzzles_toolkit_1 cat /tmp/validation.xml

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><descriptions><basics><filename>model.pdb</filename><errors><error>No. 5: Omitted record. Atom serial number is not integer (   A5). Residue sequence number is not integer ( 2A1). Unrecognized residue name (Z). X coordinate is not floating-point number (   2.A25). Y coordinate is not floating-point number ( -2A.462). Z coordinate is not floating-point number (  -3.A25).</error></errors></basics></descriptions>

In [25]:
%%bash
docker exec -i rnapuzzles_toolkit_1 /home/demo/rna-puzzles-toolkit/rnaqua-binary/bin/rnaqua.sh \
--command PDB-VALIDATION \
--single-model-file-path rna-puzzles-toolkit/rnaqua-binary/use-cases/SINGLE-STRUCTURE-ANALYSIS/PDB-VALIDATION/models/model.pdb \
--output-file-path /tmp/validation.xml

13 Oct 2018 17:59:12 : Command: PDB-VALIDATION
Base pairs identification tool: MC-ANNOTATE
Consider atoms supported by RNA-Puzzles only: N
Output file path: /tmp/validation.xml
Single model PDB file path: rna-puzzles-toolkit/rnaqua-binary/use-cases/SINGLE-STRUCTURE-ANALYSIS/PDB-VALIDATION/models/model.pdb
13 Oct 2018 17:59:12 : Command processed properly


## Clash Score

    14_ChenPostExp_1_rpr.pdb:U_1,31|U_33,29

In [6]:
%%bash
CURRENT_DIR="rna-puzzles-toolkit/rnaqua-binary/use-cases/SINGLE-STRUCTURE-ANALYSIS/SCORES/CLASH-SCORE"
docker exec -i rnapuzzles_toolkit_1 /home/demo/rna-puzzles-toolkit/rnaqua-binary/bin/rnaqua.sh --command CLASH-SCORE --single-model-file-path \
"${CURRENT_DIR}/incontinuous-models/14_ChenPostExp_1_rpr.pdb" \
--alignment "14_ChenPostExp_1_rpr.pdb:U_1,31|U_33,29" --output-file-path /tmp/14_ChenPostExp_1_rpr.xml

14 Oct 2018 17:38:19 : Command: CLASH-SCORE
Alignment: 14_ChenPostExp_1_rpr.pdb:U_1_ ,31|U_33_ ,29
Base pairs identification tool: MC-ANNOTATE
Consider atoms supported by RNA-Puzzles only: N
Output file path: /tmp/14_ChenPostExp_1_rpr.xml
Single model PDB file path: rna-puzzles-toolkit/rnaqua-binary/use-cases/SINGLE-STRUCTURE-ANALYSIS/SCORES/CLASH-SCORE/incontinuous-models/14_ChenPostExp_1_rpr.pdb
14 Oct 2018 17:38:30 : Command processed properly


In [7]:
%%bash
CURRENT_DIR="rna-puzzles-toolkit/rnaqua-binary/use-cases/SINGLE-STRUCTURE-ANALYSIS/SCORES/CLASH-SCORE"
docker exec -i rnapuzzles_toolkit_1 cat /tmp/14_ChenPostExp_1_rpr.xml
#
#<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
#<measureScores><structure><description><filename>14_ChenPostExp_1_rpr.pdb</filename><errors/></description><score>2.06</score></structure></measureScores>
#

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><measureScores><structure><description><filename>14_ChenPostExp_1_rpr.pdb</filename><errors/></description><score>2.06</score></structure></measureScores>

## Structure 3D

**ORIGINAL-3D** and **RENUMERATED-3D**

An extraction (**ORIGINAL-3D**) and unification (**RENUMERATED-3D**) of RNA 3D model(s) or a set of incontinuous 3D substructures specified by the user which are additionally superimposed over the corresponding 3D structure/substructures of the reference. At the output, ZIP archive including the coordinates of the reference structure as well as all considered RNA 3D model(s) is returned.

**Alignment**

To ensure robustness of quality assessment process a user can specify the appropriate alignment (-a,--alignment) between the reference 3D structure and all analyzed RNA 3D model(s) which often differ slightly in sequence, distribution of chains or numbering of residues. 

An example of alignment prepared between the reference structure (solution.pdb) and a single RNA 3D model (model.pdb) is presented below. This alignment considers two incontinuous 3D substructures. Moreover, there is also incompatibility of chain id between compared 3D structures. 

Each substructure is described by id of its first residue [i.e., chain id + '_' + residue serial number + '_' + insertion code (if needed)] and length. To integrate many 3D substructures within a single alignment prepared for the particular RNA 3D structure(s) their descriptions are separated by '|'. Alignments prepared for the reference structure as well as the analyzed RNA 3D models combined into a single string are separated by ';'. Alignment prepared for the reference structure should be always included at the beginning of this string.

In [3]:
%%bash
CURRENT_DIR="rna-puzzles-toolkit/rnaqua-binary/use-cases/SINGLE-STRUCTURE-ANALYSIS/STRUCTURE-3D/RENUMERATED-3D/"
docker exec -i rnapuzzles_toolkit_1 /home/demo/rna-puzzles-toolkit/rnaqua-binary/bin/rnaqua.sh \
--command RENUMERATED-3D \
--multiple-models-directory-path "${CURRENT_DIR}/models" \
--alignment "14_ChenPostExp_1_rpr.pdb:U_1,31|U_33,29" \
--output-file-path "/home/demo/14_ChenPostExp_1_rpr.zip"

14 Oct 2018 17:37:48 : Command: RENUMERATED-3D
Alignment: 14_ChenPostExp_1_rpr.pdb:U_1_ ,31|U_33_ ,29
Base pairs identification tool: MC-ANNOTATE
Consider atoms supported by RNA-Puzzles only: N
Output file path: /home/demo/14_ChenPostExp_1_rpr.zip
Multiple PDB models directory path: rna-puzzles-toolkit/rnaqua-binary/use-cases/SINGLE-STRUCTURE-ANALYSIS/STRUCTURE-3D/RENUMERATED-3D//models
14 Oct 2018 17:37:49 : Command processed properly


# rna-pdb-tools

In [ ]:
@inprogress